# Exploratory Data Analysis

In [4]:
import numpy as np
import pandas as pd
import os
import time

## Ensuring reproducibility

In [5]:
CUSTOM_SEED = 42
np.random.seed(CUSTOM_SEED)

## Read Dataset

In [6]:
path = '../dataset/Air quality/'
names =  ["No",  "year",  "month",  "day",  "hour",  "PM2.5",  "PM10",  "SO2",  "NO2",  "CO",  "O3",  "TEMP",  "PRES",  "DEWP",  "RAIN",  "wd",  "WSPM",  "station"]
dataset = pd.read_csv(path + 'Air quality.csv',  names=names)
dataset.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,5844,2013,10,30,11,41,49,19,51.1197,700,6,13.2,1022.4,-0.1,0,NE,1.4,Wanliu
1,27824,2016,5,3,7,15,26,2,11,300,72,15.5,993.5,-1.1,0,NW,3.7,Dingling
2,25841,2016,2,10,16,95,95,59,46,3100,61,9.3,1012.4,-12.6,0,ESE,1.4,Shunyi
3,26986,2016,3,29,9,10,36,12,34,500,45,15.7,1014.6,-6,0,WNW,0.8,Aotizhongxin
4,23848,2015,11,19,15,49,49,2,40,1700,2,4.6,1019.2,4,0.3,W,0.9,Wanliu


## Dataset Preparation

### Convert Column Types